# The Bristol Project 
## Model fitting
The project is framed as a multivariate regression task (for the moment). Although this may look more like a distribution of distributions which can all regress to the same answer. 

In this notebook the data will be loaded and explored 

In [1]:
import pandas as pd
import os
import historic_data_processing_functions
import logging
import betfairlightweight
from betfairlightweight import StreamListener
import bz2
import random
import numpy as np
from matplotlib import pyplot as plt

import xgboost
from xgboost import XGBRegressor, XGBClassifier
import sklearn.gaussian_process as gp
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from xgboost import XGBClassifier

from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import StandardScaler
from sklearn import svm
import seaborn as sns 
from scipy import stats
import copy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import toms_utils

/home/x4nno/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
ticks_path = "/home/x4nno/Documents/BetFair/Bristol_project/ticks.csv"
ticks_df = pd.read_csv(ticks_path)

In [3]:
# This is here so we don't have to keep loading it.

train_df1 = pd.read_csv("jan20_analysis_direct_nr0_100_50_many_wom.csv")
train_df2 = pd.read_csv("feb20_analysis_direct_nr0_100_50_many_wom.csv")
train_df3 = pd.read_csv("mar20_analysis_direct_nr0_100_50_many_wom.csv")
train_df4 = pd.read_csv("may22_analysis_direct_nr0_100_50_many_wom.csv")
train_df5 = pd.read_csv("jun22_analysis_direct_nr0_100_50_many_wom.csv")
train_df6 = pd.read_csv("jul22_analysis_direct_nr0_100_50_many_wom.csv")

frames = [train_df1, train_df2, train_df3, train_df4, train_df5, train_df6]

train_df = pd.concat(frames)
normalized_df = toms_utils.normalized_transform(train_df, ticks_df)

In [4]:
normalized_df

,bsps,mean_120,std_120,volume_120,mean_130,std_130,volume_130,mean_140,std_140,volume_140,...,WoM_ratio_900,WoM_ratio_1200,WoM_ratio_1500,WoM_ratio_1800,WoM_ratio_2700,WoM_ratio_3600,WoM_ratio_7200,WoM_ratio_14400,mean_120_temp,bsps_temp
2,1.024242,0.993939,0.009736,0.038953,1.000000,0.006553,0.018398,1.000000,0.003558,0.119546,...,1.059206,0.433724,0.545424,2.067709,2.271753,3.581497,1.408429,0.887288,3.723241,3.949456
3,0.976303,0.995261,0.000000,0.027290,0.995261,0.014742,0.086193,0.985782,0.007986,0.030455,...,2.342913,1.163452,1.397670,1.701115,1.204351,3.527649,5.342934,0.526953,10.000000,9.205764
5,0.989950,0.994975,0.011431,0.012255,0.989950,0.006966,0.075555,0.994975,0.013867,0.051094,...,1.292433,2.041370,1.871869,0.830859,1.167074,1.627799,2.283843,2.219583,7.605916,7.400000
7,0.916667,0.833333,0.004037,0.017550,0.833333,0.000732,0.080899,0.833333,0.001211,0.024824,...,0.220862,0.208113,0.222027,0.194319,0.180178,0.467605,0.184072,3.880303,1.097320,1.112750
11,0.978022,1.005495,0.008008,0.005208,1.010989,0.009343,0.027653,1.005495,0.000000,0.009092,...,0.697042,0.815562,1.340683,0.661138,0.745341,1.084937,2.359469,0.946757,5.323419,4.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4717,1.014354,0.976077,0.000000,0.014672,0.971292,0.021175,0.057792,0.966507,0.014068,0.029840,...,0.989030,0.448487,1.033406,0.799668,0.928833,0.717808,0.725698,0.791925,8.800000,10.982688
4720,1.005747,0.982759,0.006458,0.034129,0.982759,0.010492,0.070424,0.988506,0.011194,0.044691,...,1.136309,1.001094,1.214800,2.582749,2.537859,2.119100,1.018381,0.884860,4.107607,4.534631
4721,1.012658,1.000000,0.009145,0.018542,1.000000,0.003904,0.034440,0.987342,0.010239,0.043433,...,0.886478,1.175734,0.725107,0.628295,0.999214,0.737884,0.924499,1.541920,3.391078,3.486476
4723,1.004762,0.980952,0.029749,0.087544,0.985714,0.008026,0.035946,0.990476,0.011597,0.038634,...,0.471256,0.439554,0.771707,0.659437,0.919387,0.718782,1.109240,0.653562,9.287283,10.408428


In [5]:
# This whole cell is just to compare some different analysis_dfs and how well they train.

def rms(y,y_pred):
    rms=np.sqrt(np.mean((y-y_pred)**2))
    return rms
    
def train_test_classification(train_df, model):
    """Takes a dataframe (which has been preprocessed), applies train test split 
    It then predicts from the test with a confidence interval. Returns the confusion 
    matrix outputs, a green margin"""
    
    try:
        mean120_train_df = train_df["mean_120"]
        bsp_train_df = train_df["bsps"]
        if "bsps_temp" not in train_df:
            print("no bsps_temp found")
            train_df["bsps_temp"] = copy.copy(train_df["bsps"])
        train_df["bsps"] = ((mean120_train_df - bsp_train_df)>0).astype(int)

    except:
        mean300_train_df = train_df["mean_300"]
        bsp_train_df = train_df["bsps"]
        train_df["bsps_temp"] = train_df["bsps"]
        train_df["bsps"] = ((mean120_train_df - bsp_train_df)>0).astype(int)    
    
    #create two different dataframe of majority and minority class 
    df_majority = train_df[(train_df['bsps']==0)] 
    df_minority = train_df[(train_df['bsps']==1)] 
    # downsample majority
    
    df_majority = df_majority.head(len(df_minority)) # because I don't trust the resample
    # Combine majority class with upsampled minority class

    train_df = pd.concat([df_minority, df_majority])
    
    y_train_df = train_df["bsps"]
    x_train_df = train_df.drop(["bsps"], axis=1) 
    
    print("total count", y_train_df.count())
    print("total pos: ", y_train_df[y_train_df == 1].count())
    print("total neg: ", y_train_df[y_train_df == 0].count())
    
    x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(x_train_df,
                                                        y_train_df,
                                                        test_size=1/6)

    x_train_df = x_train_df.drop(["bsps_temp"], axis = 1)
    bsp_actual = x_test_df["bsps_temp"]
    x_test_df = x_test_df.drop(["bsps_temp"], axis=1)
    
    try:
        mean_120_actual = x_test_df["mean_120_temp"]
        x_test_df.drop(["mean_120_temp"], axis=1)
        x_train_df.drop(["mean_120_temp"], axis=1)
    except:
        print("no data at 120_temp")
        mean_120_actual = x_test_df["mean_120"]
        
        
#     x_test_df = x_train_df.tail(int(len(x_train_df)*(20/100)))
#     x_train_df = x_train_df.head(int(len(x_train_df)*(80/100)))
    
#     y_test_df = y_train_df.tail(int(len(y_train_df)*(20/100)))
#     y_train_df = y_train_df.head(int(len(y_train_df)*(80/100)))
    
    print("y_test count", y_test_df.count())
    print("y_test pos: ", y_test_df[y_test_df == 1].count())
    print("y_test neg: ", y_test_df[y_test_df == 0].count())

    scaler = StandardScaler()
    clm=x_train_df.columns
    x_train_df = pd.DataFrame(scaler.fit_transform(x_train_df), columns=clm)
    x_test_df = pd.DataFrame(scaler.transform(x_test_df), columns=clm)

    model.fit(x_train_df, y_train_df)

    y_pred_train = model.predict(x_train_df)
    y_pred_test = model.predict(x_test_df)

    results_train_mae = mae(y_train_df, y_pred_train)
    results_train_rms = rms(y_train_df, y_pred_train)

    print("MAE train : ", results_train_mae)
    print("RMS train : ", results_train_rms)
    
    results_test_mae = mae(y_test_df, y_pred_test)
    results_test_rms = rms(y_test_df, y_pred_test)

    print("MAE test : ", results_test_mae)
    print("RMS test : ", results_test_rms)

    (tn, fp, fn, tp) = confusion_matrix(y_test_df, y_pred_test).ravel()
    print("- tn:", tn, "- fp:", fp, "- fn:", fn, "- tp:", tp)
    
    confidence = 0.51
    print("with a confidence probability of above ", confidence)

    y_pred_train_proba = model.predict_proba(x_train_df)
    y_pred_test_proba = model.predict_proba(x_test_df)
    
    neg_pred_after_conf_train = []
    neg_corresponding_label_train = []
    
    pos_pred_after_conf_train = []
    pos_corresponding_label_train = []
    
    for i in range(len(y_pred_train_proba)):
        if y_pred_train_proba[i][0] > confidence:
            neg_pred_after_conf_train.append(0)
            neg_corresponding_label_train.append(y_train_df.values[i])
        elif y_pred_train_proba[i][1] > confidence:
            pos_pred_after_conf_train.append(1)
            pos_corresponding_label_train.append(y_train_df.values[i])
    
    print()
    results_train_mae_neg = mae(neg_pred_after_conf_train, neg_corresponding_label_train)
    results_train_mae_pos = mae(pos_pred_after_conf_train, pos_corresponding_label_train)

    print("MAE conf train neg: ", results_train_mae_neg)
    print("MAE conf train pos : ", results_train_mae_pos)
    
    neg_pred_after_conf_test = []
    neg_corresponding_label_test = []
    
    pos_pred_after_conf_test = []
    pos_corresponding_label_test = []
    
    margin = 0
    print("mean_120_actual -------------")
    print(mean_120_actual)
    print("bsps_actual ------------------")
    print(bsp_actual)
    
    for i in range(len(y_pred_test_proba)):
        if y_pred_test_proba[i][0] > confidence:
            neg_pred_after_conf_test.append(0)
            neg_corresponding_label_test.append(y_test_df.values[i])
            margin_calc = (bsp_actual.values[i] - mean_120_actual.values[i])/mean_120_actual.values[i]
            margin += margin_calc
        elif y_pred_test_proba[i][1] > confidence:
            pos_pred_after_conf_test.append(1)
            pos_corresponding_label_test.append(y_test_df.values[i])
            margin_calc = (- bsp_actual.values[i] + mean_120_actual.values[i])/mean_120_actual.values[i]
            margin += margin_calc
    
    print()
    results_test_mae_neg = mae(neg_pred_after_conf_test, neg_corresponding_label_test)
    results_test_mae_pos = mae(pos_pred_after_conf_test, pos_corresponding_label_test)

    print("MAE conf test neg: ", results_test_mae_neg)
    print("MAE conf test pos : ", results_test_mae_pos)
    
    print("confusion for neg")
    (tn, fp, fn, tp) = confusion_matrix(neg_corresponding_label_test, neg_pred_after_conf_test).ravel()
    print("- tn:", tn, "- fp:", fp, "- fn:", fn, "- tp:", tp)
    
    print("confusion for pos")
    (tn, fp, fn, tp) = confusion_matrix(pos_corresponding_label_test, pos_pred_after_conf_test).ravel()
    print("- tn:", tn, "- fp:", fp, "- fn:", fn, "- tp:", tp)
    
    print("margin for final is ", margin)
    return margin

In [6]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025), # not too bad for 300 2%
    SVC(gamma=2, C=1), # over fit
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=3),
    RandomForestClassifier(max_depth=4, n_estimators=100, max_features=4), # this is not bad
    MLPClassifier(alpha=5, max_iter=1000), # also not bad
    MLPClassifier(alpha=5, max_iter=1000, solver="adam", shuffle=True),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    XGBClassifier(colsample_bytree=0.5, subsample=0.5, tree_method="gpu_hist",
                  sampling_method="gradient_based", max_depth = 4)
]

model = classifiers[6]



############################################################################
print("------many and wom")

train_df = pd.concat(frames)
train_df = train_df.drop(["Unnamed: 0", "selection_ids", "market_id"], axis=1)
train_df = train_df.dropna()

train_df = train_df[(train_df["mean_300"] > 0) & (train_df["mean_300"] <= 50)]
train_df = train_df[train_df["mean_14400"] > 0]
train_df = train_df.drop(train_df[train_df["std_2700"] > 1].index)

train_test_classification(train_df, model)

#########################################################################
print("----- many and wom with ratio")
train_df = pd.concat(frames)
train_df = train_df.dropna()

# find wom columns 
lay_wom_list = []
back_wom_list = []
for column in train_df.columns:
    if "RWoML" in column:
        lay_wom_list.append(column)
    elif "RWoMB" in column:
        back_wom_list.append(column)

# compute ratios and add them to train_df
for i in range(len(lay_wom_list)):
    timie = lay_wom_list[i].split("_")[1]
    train_df["WoM_ratio_{}".format(timie)] = train_df[lay_wom_list[i]] / train_df[back_wom_list[i]]
    train_df = train_df.drop([lay_wom_list[i], back_wom_list[i]], axis=1)
    
train_df = train_df.drop(["Unnamed: 0", "selection_ids", "market_id"], axis=1)
train_df = train_df.dropna()
train_df = train_df[(train_df["mean_300"] > 0) & (train_df["mean_300"] <= 50)]
train_df = train_df[train_df["mean_14400"] > 0]
train_df = train_df.drop(train_df[train_df["std_2700"] > 1].index)

train_test_classification(train_df, model)

############################################################################
print("--")
print("normalized values")
print("--")
# # lets now try using ticks and total average? so mean_ticks / total_mean_ticks
normalized_df_feed = copy.copy(normalized_df) # so we don't change the one which takes ages
train_test_classification(normalized_df_feed, model)


train_df.head()
# Note for after - we should standardise the volumes and the stds ... 

------many and wom
no bsps_temp found
total count 10678
total pos:  5339
total neg:  5339
no data at 120_temp
y_test count 1780
y_test pos:  869
y_test neg:  911
MAE train :  0.46313778377163406
RMS train :  0.6805422718477039
MAE test :  0.4893258426966292
RMS test :  0.6995182933252205
- tn: 440 - fp: 471 - fn: 400 - tp: 469
with a confidence probability of above  0.51

MAE conf train neg:  0.4231750893312915
MAE conf train pos :  0.4491494752081071
mean_120_actual -------------
1235     2.578712
781     16.422201
3962    11.500000
1816     7.698600
2511     4.194629
          ...    
1265     2.099409
1329    15.664659
2544     9.383395
2656     5.012473
2010     8.200000
Name: mean_120, Length: 1780, dtype: float64
bsps_actual ------------------
1235     2.897421
781     15.399269
3962    11.000000
1816     5.900000
2511     4.128829
          ...    
1265     2.319335
1329    18.000000
2544    16.941948
2656     5.739461
2010     8.782097
Name: bsps_temp, Length: 1780, dtype: floa

,bsps,mean_120,std_120,volume_120,mean_130,std_130,volume_130,mean_140,std_140,volume_140,...,WoM_ratio_800,WoM_ratio_900,WoM_ratio_1200,WoM_ratio_1500,WoM_ratio_1800,WoM_ratio_2700,WoM_ratio_3600,WoM_ratio_7200,WoM_ratio_14400,bsps_temp
2,0,3.723241,0.036249,1373.55,3.771105,0.024714,648.76,3.753867,0.013357,4215.45,...,0.714384,1.059206,0.433724,0.545424,2.067709,2.271753,3.581497,1.408429,0.887288,3.949456
3,1,10.000000,0.000000,231.91,10.048542,0.148137,732.46,9.564714,0.076385,258.80,...,1.247266,2.342913,1.163452,1.397670,1.701115,1.204351,3.527649,5.342934,0.526953,9.205764
5,1,7.605916,0.086941,128.47,7.414352,0.051651,792.06,7.538499,0.104537,535.63,...,1.467594,1.292433,2.041370,1.871869,0.830859,1.167074,1.627799,2.283843,2.219583,7.400000
7,0,1.097320,0.004430,4151.88,1.099935,0.000805,19139.24,1.099820,0.001331,5872.87,...,0.302111,0.220862,0.208113,0.222027,0.194319,0.180178,0.467605,0.184072,3.880303,1.112750
11,1,5.323419,0.042627,76.86,5.352315,0.050008,408.07,5.300000,0.000000,134.17,...,0.951805,0.697042,0.815562,1.340683,0.661138,0.745341,1.084937,2.359469,0.946757,4.800000


In [7]:
# run a cross val to check our values.
model = classifiers[6]
cross_val_margin = []
for i in range(30):
    normalized_df_feed = copy.copy(normalized_df) # don't change this each time
    cross_val_margin.append(train_test_classification(normalized_df_feed, model))

total count 10054
total pos:  5027
total neg:  5027
y_test count 1676
y_test pos:  840
y_test neg:  836
MAE train :  0.401527810933397
RMS train :  0.6336622214819162
MAE test :  0.4904534606205251
RMS test :  0.7003238255411028
- tn: 433 - fp: 403 - fn: 419 - tp: 421
with a confidence probability of above  0.51

MAE conf train neg:  0.3878984332793085
MAE conf train pos :  0.38769906644700713
mean_120_actual -------------
544      4.176939
720      8.400000
2083    20.701252
4265    12.078551
2824     4.080788
          ...    
1850     6.400000
89       4.619543
2255     1.827554
1059     6.633324
2192     4.198382
Name: mean_120_temp, Length: 1676, dtype: float64
bsps_actual ------------------
544      4.200000
720      9.674618
2083    15.479228
4265    13.500000
2824     4.700000
          ...    
1850     6.200000
89       4.880177
2255     1.870271
1059     6.600000
2192     5.056321
Name: bsps_temp, Length: 1676, dtype: float64

MAE conf test neg:  0.48998664886515353
MAE conf 

MAE train :  0.40534733826688946
RMS train :  0.6366689392980385
MAE test :  0.45346062052505964
RMS test :  0.6733948474149914
- tn: 435 - fp: 407 - fn: 353 - tp: 481
with a confidence probability of above  0.51

MAE conf train neg:  0.3830357142857143
MAE conf train pos :  0.3943326271186441
mean_120_actual -------------
1612     3.807177
2611     8.202143
2658    20.007752
2122    10.031746
365      1.783903
          ...    
925      2.850936
3906     3.878098
1613    22.084189
4745     4.971477
1079     8.088824
Name: mean_120_temp, Length: 1676, dtype: float64
bsps_actual ------------------
1612     3.900000
2611     6.331366
2658     8.549723
2122     9.545025
365      1.620000
          ...    
925      2.867497
3906     3.479984
1613    20.000000
4745     4.665815
1079     7.468807
Name: bsps_temp, Length: 1676, dtype: float64

MAE conf test neg:  0.43815201192250375
MAE conf test pos :  0.4476067270375162
confusion for neg
- tn: 377 - fp: 0 - fn: 294 - tp: 0
confusion for pos

MAE train :  0.4064215803294342
RMS train :  0.6375120236744043
MAE test :  0.46897374701670647
RMS test :  0.6848165791047311
- tn: 466 - fp: 359 - fn: 427 - tp: 424
with a confidence probability of above  0.51

MAE conf train neg:  0.39771250324928514
MAE conf train pos :  0.38746690203000883
mean_120_actual -------------
444     18.500000
1902     8.666024
24      21.284464
900      5.124624
3073     1.670691
          ...    
852     36.941176
235     19.594046
732      6.800000
87      31.284155
2259     4.100000
Name: mean_120_temp, Length: 1676, dtype: float64
bsps_actual ------------------
444     17.634700
1902     6.709705
24      26.000000
900      5.200000
3073     1.630000
          ...    
852     30.000000
235     18.000000
732      6.737124
87      43.858073
2259     5.382085
Name: bsps_temp, Length: 1676, dtype: float64

MAE conf test neg:  0.46846846846846846
MAE conf test pos :  0.43833580980683506
confusion for neg
- tn: 413 - fp: 0 - fn: 364 - tp: 0
confusion for p

MAE train :  0.4067796610169492
RMS train :  0.6377928041432807
MAE test :  0.4510739856801909
RMS test :  0.671620417855347
- tn: 483 - fp: 332 - fn: 424 - tp: 437
with a confidence probability of above  0.51

MAE conf train neg:  0.3981599795553284
MAE conf train pos :  0.390281173594132
mean_120_actual -------------
1633     4.315486
1667     6.352342
465      5.381602
1408     5.576501
3676     1.785341
          ...    
4167     8.288988
909      2.662583
1347     3.470828
2437     1.880276
831     32.000000
Name: mean_120_temp, Length: 1676, dtype: float64
bsps_actual ------------------
1633     4.538112
1667     6.153400
465      5.924365
1408     5.930590
3676     1.730239
          ...    
4167     5.800000
909      2.488797
1347     3.772494
2437     1.751516
831     38.662979
Name: bsps_temp, Length: 1676, dtype: float64

MAE conf test neg:  0.47058823529411764
MAE conf test pos :  0.4199066874027994
confusion for neg
- tn: 414 - fp: 0 - fn: 368 - tp: 0
confusion for pos
- t

MAE train :  0.40701838147529246
RMS train :  0.6379799224703646
MAE test :  0.47076372315035797
RMS test :  0.6861222363036764
- tn: 438 - fp: 407 - fn: 382 - tp: 449
with a confidence probability of above  0.51

MAE conf train neg:  0.3968603690443404
MAE conf train pos :  0.3946847610969748
mean_120_actual -------------
702     14.852607
1090     9.691352
3174    11.460228
1065     8.200000
34       2.786184
          ...    
1296     6.000000
3956     5.400005
4290     3.008340
1135    12.000000
1520    24.535094
Name: mean_120_temp, Length: 1676, dtype: float64
bsps_actual ------------------
702     15.484184
1090     8.079428
3174     7.200000
1065     7.421571
34       2.674329
          ...    
1296     6.475675
3956     5.577852
4290     2.800000
1135    13.654379
1520    25.000000
Name: bsps_temp, Length: 1676, dtype: float64

MAE conf test neg:  0.4624277456647399
MAE conf test pos :  0.4604715672676838
confusion for neg
- tn: 372 - fp: 0 - fn: 320 - tp: 0
confusion for pos


In [8]:
cross_val_margin

[13.45799712110817,
 26.23715305717762,
 20.750332845292004,
 27.210453519413324,
 34.09019420015518,
 24.949850468122843,
 28.69297647518651,
 34.85254769435564,
 17.428936019462512,
 38.6650446060163,
 20.3211923936006,
 20.685450080219574,
 25.28363775358057,
 15.221416911531227,
 35.183721034731356,
 15.810764791001983,
 28.155006190240474,
 27.04095092522561,
 22.700450787076733,
 38.59725995144449,
 21.23542342919315,
 28.804684399193327,
 26.898856710872824,
 27.36045283995594,
 32.627117587076945,
 22.454016933585507,
 22.583657510184878,
 26.668189204789858,
 18.88310558763496,
 22.627402883923555]

In [9]:
sum(cross_val_margin)/len(cross_val_margin)

25.515941463711787